# Extract Tags
This is a notebook to extract images from a ROS 1 bagfile, detect the aruco markers, and output a bag of AprilTag detections.

In [40]:
#!/usr/bin/env python3

from rosbags.highlevel import AnyReader
from rosbags.image import message_to_cvimage
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2
from apriltag_msgs.msg import ApriltagArrayStamped, Apriltag
from geometry_msgs.msg import Point
import rospy
import rosbag

# Set up paths
bag_file_path = '/home/asekar/Documents/trisect_ws/src/trisect_data/2023-07-21_head_orbit/cropped.bag'
output_bag_path = '/home/asekar/test/fiducial_ws/src/aruco_to_apriltag/output/apriltag_output.bag'
image_topic = '/trisect/stereo/left/image_rect'

In [41]:
def extract_and_generate_apriltag_msgs(bag_file_path, image_topic, encoding="bgr8"):

    # Set up the ArUco dictionary and parameters for detection
    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)
    parameters = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

    out_bag = rosbag.Bag(output_bag_path, 'w')  # Open output bag for writing

    # Open the bag file with AnyReader
    with AnyReader([Path(bag_file_path)]) as reader:
        connections = [x for x in reader.connections if x.topic == image_topic]
        
        for connection, timestamp, rawdata in reader.messages(connections=connections):
            # Deserialize the message
            msg = reader.deserialize(rawdata, connection.msgtype)
            
            # Convert ROS message to OpenCV image
            cv_image = message_to_cvimage(msg, encoding)
            
            # Detect ArUco markers
            corners, ids, rejected = detector.detectMarkers(cv_image)
            
            # Draw detected markers on the image
            if ids is not None:
                cv_image = cv2.aruco.drawDetectedMarkers(cv_image, corners, ids)

                # Create the ApriltagArrayStamped message
                apriltag_array_msg = ApriltagArrayStamped()
                apriltag_array_msg.header.stamp = msg.header.stamp  
                apriltag_array_msg.header.frame_id = "camera_frame"  # Update this to match your frame

                # Loop over detected ArUco markers to populate ApriltagArrayStamped
                for i, corner in enumerate(corners):
                    apriltag_msg = Apriltag()
                    apriltag_msg.id = int(ids[i])
                    apriltag_msg.family = "4X4_250"
                    
                    # Calculate the center point as the average of the corners
                    center_x = sum([c[0] for c in corner[0]]) / 4.0
                    center_y = sum([c[1] for c in corner[0]]) / 4.0
                    apriltag_msg.center = Point(center_x, center_y, 0)

                    # Set the corners in the apriltag_msg
                    apriltag_msg.corners = [
                        Point(c[0], c[1], 0) for c in corner[0]
                    ]

                    # Add to the array message
                    apriltag_array_msg.apriltags.append(apriltag_msg)
                
                # Convert to rospy.Time explicitly
                ros_time = rospy.Time(msg.header.stamp.sec, msg.header.stamp.nanosec)
                
                # Assign this rospy.Time object to apriltag_array_msg.header.stamp
                apriltag_array_msg.header.stamp = ros_time
                
                # Write the message with the converted timestamp
                out_bag.write('/apriltag_detections', apriltag_array_msg, t=ros_time)
                # print(f"Written ApriltagArrayStamped message at time: {ros_time.to_sec()}")
            
            # # Display the image with detected markers
            # plt.imshow(cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB))
            # plt.axis('off')
            # plt.show()

    out_bag.close()
    print(f"Output bag written to: {output_bag_path}")


In [42]:
extract_and_generate_apriltag_msgs(bag_file_path, image_topic)

Output bag written to: /home/asekar/test/fiducial_ws/src/aruco_to_apriltag/output/apriltag_output.bag
